In [1]:
!pip install /data/snowflake_ml_python-0.2.3-py3-none-any.whl

Processing /data/snowflake_ml_python-0.2.3-py3-none-any.whl
     |████████████████████████████████| 143kB 9.9MB/s eta 0:00:01
     |████████████████████████████████| 706kB 93.8MB/s eta 0:00:01
     |████████████████████████████████| 17.1MB 73.6MB/s eta 0:00:01
     |████████████████████████████████| 34.5MB 98.3MB/s eta 0:00:01
     |████████████████████████████████| 276kB 70.8MB/s eta 0:00:01
     |████████████████████████████████| 133kB 100.1MB/s eta 0:00:01
     |████████████████████████████████| 9.8MB 76.4MB/s eta 0:00:01
     |████████████████████████████████| 51kB 757kB/s  eta 0:00:01
     |████████████████████████████████| 193.6MB 32kB/s /s eta 0:00:01
     |████████████████████████████████| 24.6MB 97.0MB/s eta 0:00:01
     |████████████████████████████████| 1.0MB 66.9MB/s eta 0:00:01
     |████████████████████████████████| 71kB 6.4MB/s  eta 0:00:01
     |████████████████████████████████| 1.1MB 86.2MB/s eta 0:00:01
     |████████████████████████████████| 71kB 4.3MB/s  eta 0:00:01

     |████████████████████████████████| 143kB 107.8MB/s eta 0:00:01
     |████████████████████████████████| 512kB 109.3MB/s eta 0:00:01
     |████████████████████████████████| 12.3MB 96.6MB/s eta 0:00:01
     |████████████████████████████████| 36.0MB 103.6MB/s eta 0:00:01
     |████████████████████████████████| 61kB 5.4MB/s  eta 0:00:01
     |████████████████████████████████| 276kB 101.1MB/s eta 0:00:01
     |████████████████████████████████| 163kB 94.0MB/s eta 0:00:01
     |████████████████████████████████| 122kB 97.6MB/s eta 0:00:01
     |████████████████████████████████| 122kB 106.1MB/s eta 0:00:01
     |████████████████████████████████| 256kB 93.8MB/s eta 0:00:01
     |████████████████████████████████| 348kB 96.7MB/s eta 0:00:01
ERROR: jupyterlab 3.2.4 has requirement jupyter-server~=1.4, but you'll have jupyter-server 2.0.0a1 which is incompatible.
ERROR: jupyterlab-server 2.22.1 has requirement jsonschema>=4.17.3, but you'll have jsonschema 3.2.0 which is incompatible.


In [2]:
# !pip install seaborn
# !pip install matplotlib
! pip install pandas==1.5.3

  Using cached https://files.pythonhosted.org/packages/54/a0/c62d63c5c69be9aae07836e4d7e25e7a6f5590be3d8f2d53f43eeec5c475/pandas-1.5.3-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/8b/d9/814a619ab84d8eb0d95e08d4c723e665f1e694b5a6068ca505a61bdc3745/numpy-1.24.3-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/7f/99/ad6bd37e748257dd70d6f85d916cafe79c0b0f5e2e95b11f7fbc82bf3110/pytz-2023.3-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/36/7a/87837f39d0296e723bb9b62bbb257d0355c7f6128853c78955f57342a56d/python_dateutil-2.8.2-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/d9/5a/e7c31adbe875f2abbb91bd84cf2dc52d792b5a01506781dbcf25c91daf11/six-1.16.0-py2.py3-none-any.whl
ERROR: snowflake-ml-python 0.2.3 has requirement numpy<1.24,>=1.23, but you'll have numpy 1.24.3 which is incompatible.


In [1]:
import pandas as pd
pd.__version__

'2.0.1'

In [1]:
#  Snowpark Connector, Snowpark Library, Session
import snowflake.connector
import snowflake.snowpark
import snowflake.ml.preprocessing as snowml
from snowflake.snowpark import Session
from snowflake.snowpark.version import VERSION
from snowflake.ml.utils import connection_params
from snowflake.ml.metrics.correlation import correlation

/tmp/pip_packages/snowflake/connector/options.py:108: UserWarning: You have an incompatible version of 'pyarrow' installed (10.0.1), please install a version that adheres to: 'pyarrow<8.1.0,>=8.0.0; extra == "pandas"'
  warn_incompatible_dep(


In [3]:
from snowflake.ml.framework.pipeline import Pipeline
from snowflake.ml.modeling.xgboost import XGBClassifier
from snowflake.ml.preprocessing import MinMaxScaler, OrdinalEncoder, OneHotEncoder
# import seaborn as sns
from sklearn.metrics import mean_absolute_percentage_error

In [4]:
# Pandas Tools
from snowflake.connector.pandas_tools import write_pandas
# Data Science Libs
import numpy as np
import pandas as pd
# create_temp_table warning suppresion
import warnings; warnings.simplefilter('ignore')

In [5]:
import configparser
def get_session():
    parser = configparser.ConfigParser()
    # Add the credential file name here
    parser.read('/notebooks/notebooks/config.ini')

    connection_params = dict(user=parser['Credentials']['user'], 
                         password=parser['Credentials']['password'], 
                         account=parser['Credentials']['account'], 
                         warehouse=parser['Credentials']['warehouse'], 
                         database=parser['Credentials']['database'],
                         schema=parser['Credentials']['schema'], 
                         role=parser['Credentials']['role'])

    session = Session.builder.configs(connection_params).create()
    return session



In [6]:
session = get_session()
# session.close()
session


In [7]:
session.get_current_role()

'"ACCOUNTADMIN"'

# Model Building

In [8]:
table_name = 'BANK_CUSTOMER_CHURN_PREDICTION'

### Bank Customer Churn Dataset
Bank Customer Data for Predicting Customer Churn 

<b>About Dataset</b>

This dataset is for ABC Multistate bank with following columns:

    customer_id, unused variable.
    credit_score, used as input.
    country, used as input.
    gender, used as input.
    age, used as input.
    tenure, used as input.
    balance, used as input.
    products_number, used as input.
    credit_card, used as input.
    active_member, used as input.
    estimated_salary, used as input.
    churn, used as the target. 1 if the client has left the bank during some period or 0 if he/she has not.
    
https://www.kaggle.com/datasets/gauravtopre/bank-customer-churn-dataset


In [9]:
# # Define the Table and Cleanup Cols, have a work_schema for testing
# ############################################################################
# # NOTE:
# #
# # Set work_schema variable to some schema that exists in your account.
# #
# # set data_dir to point to the directory that contains the diamonds.csv f
# ############################################################################
# # Read csv of diamonds data to a dataframe, set data_dir to path of your csv
# df = pd.read_csv("/data/bank_customer_churn_prediction.csv")
# # Cleanup - remove row column and force headers to Uppercase
# df.columns = df.columns.str.upper()
# df.rename(columns={'TABLE': 'TABLE_PCT'}, inplace=True)
# # write the DF to Snowflake and create a Snowflake DF
# session.write_pandas(df, table_name, auto_create_table=True)

## Load data to snowflake dataframe

In [10]:
sf_df = session.table(table_name).drop('ROW')

In [11]:
sf_df.show(3)

-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"CUSTOMER_ID"  |"CREDIT_SCORE"  |"COUNTRY"  |"GENDER"  |"AGE"  |"TENURE"  |"BALANCE"  |"PRODUCTS_NUMBER"  |"CREDIT_CARD"  |"ACTIVE_MEMBER"  |"ESTIMATED_SALARY"  |"CHURN"  |
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|15634602       |619             |France     |Female    |42     |2         |0.0        |1                  |1              |1                |101348.88           |1        |
|15647311       |608             |Spain      |Female    |41     |1         |83807.86   |1                  |0              |1                |112542.58           |0        |
|15619304       |502             |France     |Female    |42     |8         |159660.8   |3                  |1              |0     

In [12]:
CATEGORICAL_COLUMNS = ["COUNTRY", "GENDER"]
CATEGORICAL_COLUMNS_OE = ["COUNTRY_OE", "GENDER_OE"]
NUMERICAL_COLUMNS = ["CREDIT_SCORE", "AGE", "TENURE", "BALANCE", "PRODUCTS_NUMBER", "ESTIMATED_SALARY"]
LABEL_COLUMNS = ["CHURN"]
OUTPUT_COLUMNS = ["PREDICTION"]

1)Create temp table from base table with one extra column  : temp creates a table for this session only

<i>CREATE  SCOPED TEMPORARY  TABLE SNOWPARK_TEMP_TABLE_V49HYUD3TS("CUSTOMER_ID" BIGINT, "CREDIT_SCORE" BIGINT, "COUNTRY" STRING, "GENDER" STRING, "AGE" BIGINT, "TENURE" BIGINT, "BALANCE" DOUBLE, "PRODUCTS_NUMBER" BIGINT, "CREDIT_CARD" BIGINT, "ACTIVE_MEMBER" BIGINT, "ESTIMATED_SALARY" DOUBLE, "CHURN" BIGINT, "SNOWPARK_TEMP_COLUMN_EQWFD9TDSN" BIGINT)</i>

2)Insert random integers into the new column create in the temp table

<i>INSERT  INTO SNOWPARK_TEMP_TABLE_V49HYUD3TS SELECT  *  FROM ( SELECT "CUSTOMER_ID", "CREDIT_SCORE", "COUNTRY", "GENDER", "AGE", "TENURE", "BALANCE", "PRODUCTS_NUMBER", "CREDIT_CARD", "ACTIVE_MEMBER", "ESTIMATED_SALARY", "CHURN", (abs(random(6109115594032942777)) % 1000000 :: INT) AS "SNOWPARK_TEMP_COLUMN_EQWFD9TDSN" FROM BANK_CUSTOMER_CHURN_PREDICTION)</i>


In [13]:
train_df, test_df = sf_df.random_split([0.8,0.2])

<i>SELECT "CUSTOMER_ID", "CREDIT_SCORE", "COUNTRY", "GENDER", "AGE", "TENURE", "BALANCE", "PRODUCTS_NUMBER", "CREDIT_CARD", "ACTIVE_MEMBER", "ESTIMATED_SALARY", "CHURN" FROM ( SELECT  *  FROM SNOWPARK_TEMP_TABLE_V49HYUD3TS WHERE (("SNOWPARK_TEMP_COLUMN_EQWFD9TDSN" >= 0 :: INT) AND ("SNOWPARK_TEMP_COLUMN_EQWFD9TDSN" < 800000 :: INT))) LIMIT 3 </i>

In [14]:
train_df.show(3)

-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"CUSTOMER_ID"  |"CREDIT_SCORE"  |"COUNTRY"  |"GENDER"  |"AGE"  |"TENURE"  |"BALANCE"  |"PRODUCTS_NUMBER"  |"CREDIT_CARD"  |"ACTIVE_MEMBER"  |"ESTIMATED_SALARY"  |"CHURN"  |
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|15634602       |619             |France     |Female    |42     |2         |0.0        |1                  |1              |1                |101348.88           |1        |
|15619304       |502             |France     |Female    |42     |8         |159660.8   |3                  |1              |0                |113931.57           |1        |
|15701354       |699             |France     |Female    |39     |1         |0.0        |2                  |0              |0     

In [15]:
pipeline = Pipeline(
    steps=[
            (
                "OE",
                OrdinalEncoder(
                input_cols=CATEGORICAL_COLUMNS,
                output_cols=CATEGORICAL_COLUMNS_OE
                )
            ),
            (
                "MMS",
                MinMaxScaler(
                clip=True,
                input_cols=NUMERICAL_COLUMNS,
                output_cols=NUMERICAL_COLUMNS,
                )
            ),
            (
                "classification",
                XGBClassifier(
#                 input_cols=NUMERICAL_COLUMNS,
                input_cols=CATEGORICAL_COLUMNS_OE+NUMERICAL_COLUMNS + ["CREDIT_CARD"],
                label_cols=LABEL_COLUMNS,
                output_cols=OUTPUT_COLUMNS
                )
            )
    ]
)

In [17]:
# # Train
pipeline.fit(train_df)

RuntimeError: Package pandas==2.0.2 is not supported in snowflake conda channel for python runtime 3.8.

In [16]:
pd.__version__

'2.0.2'

In [16]:
# # Eval
result = pipeline.predict(test_df)

In [17]:
#get 10 records and dispal
result.show()

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"CUSTOMER_ID"  |"CREDIT_SCORE"       |"GENDER_OE"  |"ESTIMATED_SALARY"   |"ACTIVE_MEMBER"  |"COUNTRY"  |"CHURN"  |"GENDER"  |"AGE"                |"BALANCE"            |"CREDIT_CARD"  |"COUNTRY_OE"  |"PRODUCTS_NUMBER"   |"TENURE"            |"PREDICTION"  |
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|15574012       |0.59                 |1.0          |0.7486985274464675   |0                |Spain      |1        |Male      |0.35135135135135137  |0.47718840697895476  |1              |2.0           |0.3333333333333333  |0

### Savind model/ pipeline in refract NAS

In [18]:
from snowflake.ml.model import model as mdl, model_signature

In [19]:
skl = pipeline.get_sklearn_object()

In [20]:
from typing import Sequence
import snowflake.snowpark.types as spt
import numpy

In [21]:
model_signature.DataType.DOUBLE.value

('double', DoubleType(), numpy.float64)

In [22]:
s1 = [
        model_signature.FeatureSpec(
            name="input_array",
            dtype=('double', spt.DoubleType(), numpy.float64)
        )
    ]

In [23]:
sign = model_signature.ModelSignature(
    inputs=[
        model_signature.FeatureSpec(
            name="input_array",
            dtype=model_signature.DataType.DOUBLE
        )
    ],
    outputs=[
        model_signature.FeatureSpec(
            name="output_array",
            dtype=model_signature.DataType.INT64
        )
    ]
)

In [122]:
mdl.save_model(name="churn.model",model_dir_path="/data/models",model=skl,signature=sign)

### Pulling only ground truth and prediction out of snowflake
Even this step can be pushed to snowflake as a UDF

In [24]:
out_df = result["CHURN", "PREDICTION"].to_pandas()

In [25]:
y_true = out_df["CHURN"]
y_pred = out_df["PREDICTION"]

In [26]:
#to make the datatype of values in both column consistant
y_pred = y_pred.astype(y_true.dtype)

In [27]:
# # example of a ordinal encoding
# from numpy import asarray
# from sklearn.preprocessing import OrdinalEncoder
# # define data

# # define ordinal encoding
# encoder = OrdinalEncoder()
# df[["GENDER"]] = encoder.fit_transform(df[["GENDER"]])
# # d data
# # result = encoder.fit_transform(train_df[["GENDER"]])
# print(df[["GENDER"]])

In [28]:
out_df.head(8)

,CHURN,PREDICTION
0,1,0
1,0,0
2,0,0
3,0,0
4,0,0
5,1,0
6,0,1
7,1,0


In [29]:
# from snowflake.ml.metrics import classification_report
from sklearn.metrics import confusion_matrix,log_loss,classification_report

In [30]:
target_names = ["stay", "exit"]

In [31]:
report = classification_report(y_true,y_pred,target_names=target_names, output_dict=True)

In [32]:
report

{'stay': {'precision': 0.8797250859106529,
  'recall': 0.9360146252285192,
  'f1-score': 0.9069973427812222,
  'support': 1641},
 'exit': {'precision': 0.5914396887159533,
  'recall': 0.4198895027624309,
  'f1-score': 0.4911147011308562,
  'support': 362},
 'accuracy': 0.8427358961557664,
 'macro avg': {'precision': 0.7355823873133032,
  'recall': 0.677952063995475,
  'f1-score': 0.6990560219560392,
  'support': 2003},
 'weighted avg': {'precision': 0.8276235812753652,
  'recall': 0.8427358961557664,
  'f1-score': 0.83183532766518,
  'support': 2003}}

# Model Registry

In [33]:
from snowflake.ml.utils.connection_params import SnowflakeLoginOptions
from snowflake.snowpark import Session, Column, functions

In [32]:
# session = Session.builder.configs(SnowflakeLoginOptions()).create()
session = get_session()

In [34]:
import importlib
from snowflake.ml.registry import model_registry
importlib.reload(model_registry)

<module 'snowflake.ml.registry.model_registry' from '/tmp/pip_packages/snowflake/ml/registry/model_registry.py'>

In [41]:
create_result = model_registry.create_model_registry(session=session,database_name="MODEL_REGISTRY_SNOWFRACT")

In [18]:
#create new model registry. This will be a no-op if the registry already exists
create_result = model_registry.create_model_registry(session)

In [42]:
registry = model_registry.ModelRegistry(session=session, name="MODEL_REGISTRY")

In [43]:
registry

Registering a new model is always done using relational API
The call to log_model executes a few steps:
1. The given model object is serialized and uploaded to a stage
2. An entry in the model registry is created for the model, referencing the model stage location.
3. Additional metadata is updated for the model as provided in the call.

For serialization to work, the model object needs to be serializable in python

<!-- Signature:
registry.log_model(
    *,
    model: Any,
    signature: snowflake.ml.model.model_signature.ModelSignature,
    name: Union[str, NoneType] = None,
    tags: Union[Dict[str, str], NoneType] = None,
    pip_requirements: Union[List[str], NoneType] = None,
    sample_input_data: Union[Any, NoneType] = None,
) -> Union[str, NoneType]
Docstring:
Uploads a model to the Model Registry.

Args:
    model: Local model object in a supported format.
    signature: Signature of the model, could be inferred by calling `infer_signature` method with sample
        input data.
    name: The given name for the model.
    tags: string-to-string dictonary of tag names and values to be set for the model.
    pip_requirements: List of dependencies to include with the model
    sample_input_data: Sample of the input data for the model.

Returns:
    String of the auto-generate unique model identifier. None if failed.
File:      /tmp/pip_packages/snowflake/ml/registry/model_registry.py
Type:      method -->

In [94]:
# a name and model tags can be added to the model at the registration time
model_id = registry.log_model(model=pipeline,name="Churn Prediction 19th May",signature=sign, tags={
    "stage": "testing","classifier_type":"xgb"
})

### Load model from snowflake repo

In [95]:
# need model id for this
ppln_loaded = model_registry.ModelReference(registry=registry, id=model_id)

In [96]:
type(ppln_loaded)

snowflake.ml.registry.model_registry.ModelReference

In [97]:
model_id

'bad9c44af62611ed95000a3279fc73b5'

## List models form snowflake repository

In [98]:
# this returns a snowpark dataframe
model_list = registry.list_models()

In [99]:
model_list.to_pandas()

,CREATION_CONTEXT,CREATION_ENVIRONMENT_SPEC,CREATION_ROLE,CREATION_TIME,ID,INPUT_SPEC,OUTPUT_SPEC,RUNTIME_ENVIRONMENT_SPEC,TYPE,URI,VERSION,DESCRIPTION,METRICS,NAME,TAGS,REGISTRATION_TIMESTAMP
0,None,"{\n ""python"": ""3.8.13""\n}","""ACCOUNTADMIN""",2023-05-14 21:05:06.402000-07:00,ac7b0372f2d511ed9bd81a58bc4a8d4c,None,None,None,Pipeline,sfc:MODEL_REGISTRY.PUBLIC.SNOWML_MODEL_AC7B037...,None,None,"{\n ""classification_report"": {\n ""0"": {\n ...","""Diabeties detector""","{\n ""classifier_type"": ""xgb"",\n ""stage"": ""te...",2023-05-14 21:05:09.854000-07:00
1,None,"{\n ""python"": ""3.8.13""\n}","""ACCOUNTADMIN""",2023-05-15 03:36:44.798000-07:00,62b26578f30c11ed877cba735c640ef5,None,None,None,Pipeline,sfc:MODEL_REGISTRY.PUBLIC.SNOWML_MODEL_62B2657...,None,None,"{\n ""classification_report"": {\n ""accuracy...","""Diabeties detector""","{\n ""classifier_type"": ""xgb"",\n ""stage"": ""te...",2023-05-15 03:36:46.579000-07:00
2,None,"{\n ""python"": ""3.8.13""\n}","""ACCOUNTADMIN""",2023-05-05 03:59:15.005000-07:00,df39f65aeb3311edb54d8a8aaf1a29c2,None,None,None,Pipeline,sfc:MODEL_REGISTRY.PUBLIC.SNOWML_MODEL_DF39F65...,None,None,None,"""Diabeties detector""","{\n ""classifier_type"": ""xgb"",\n ""stage"": ""te...",2023-05-05 03:59:18.195000-07:00
3,None,"{\n ""python"": ""3.8.13""\n}","""ACCOUNTADMIN""",2023-05-15 03:14:06.021000-07:00,38dda12af30911ed82411a58bc4a8d4c,None,None,None,Pipeline,sfc:MODEL_REGISTRY.PUBLIC.SNOWML_MODEL_38DDA12...,None,None,"{\n ""classification_report"": {\n ""accuracy...","""Diabeties detector""","{\n ""classifier_type"": ""xgb"",\n ""stage"": ""te...",2023-05-15 03:14:07.916000-07:00
4,None,"{\n ""python"": ""3.8.13""\n}","""ACCOUNTADMIN""",2023-05-19 02:06:34.748000-07:00,73ca7a60f62411ed95000a3279fc73b5,None,None,None,Pipeline,sfc:MODEL_REGISTRY.PUBLIC.SNOWML_MODEL_73CA7A6...,None,None,None,"""Diabeties detector 19th May""","{\n ""classifier_type"": ""xgb"",\n ""stage"": ""te...",2023-05-19 02:06:37.488000-07:00
5,None,"{\n ""python"": ""3.8.13""\n}","""ACCOUNTADMIN""",2023-05-19 02:11:25.920000-07:00,20fb2b08f62511ed95000a3279fc73b5,None,None,None,Pipeline,sfc:MODEL_REGISTRY.PUBLIC.SNOWML_MODEL_20FB2B0...,None,None,"{\n ""classification_report"": {\n ""accuracy...","""Diabeties detector 19th May""","{\n ""classifier_type"": ""xgb"",\n ""stage"": ""te...",2023-05-19 02:11:27.365000-07:00
6,None,"{\n ""python"": ""3.8.13""\n}","""ACCOUNTADMIN""",2023-05-19 02:22:52.971000-07:00,bad9c44af62611ed95000a3279fc73b5,None,None,None,Pipeline,sfc:MODEL_REGISTRY.PUBLIC.SNOWML_MODEL_BAD9C44...,None,None,None,"""Churn Prediction 19th May""","{\n ""classifier_type"": ""xgb"",\n ""stage"": ""te...",2023-05-19 02:22:54.607000-07:00


In [100]:
model_list.filter(model_list["ID"] == model_id).select("NAME","TAGS","METRICS").show()

-------------------------------------------------------------------------
|"NAME"                       |"TAGS"                       |"METRICS"  |
-------------------------------------------------------------------------
|"Churn Prediction 19th May"  |{                            |NULL       |
|                             |  "classifier_type": "xgb",  |           |
|                             |  "stage": "testing"         |           |
|                             |}                            |           |
-------------------------------------------------------------------------



## Adding metrics to a model registered

In [101]:
registry.set_metric(id=model_id,name="classification_report", value=report)

In [102]:
logloss = log_loss(y_true,y_pred,normalize=True)

In [103]:
registry.set_metric(id=model_id,name="log_loss", value=logloss)

------------------------------------------------------------------------------------------------
<b>Adding confusion matrix as metrix</b>

In [104]:
cnf_mtr = confusion_matrix(y_true=y_true,y_pred=y_pred)

In [105]:
cnf_mtr

array([[1536,  105],
       [ 210,  152]])

In [106]:
# is numpy array as metics possible?
registry.set_metric(id=model_id,name="confusion_matrix", value=cnf_mtr)

In [107]:
model_list.filter(model_list["ID"] == model_id).select("NAME","TAGS","METRICS").show()

------------------------------------------------------------------------------------------------------
|"NAME"                       |"TAGS"                       |"METRICS"                               |
------------------------------------------------------------------------------------------------------
|"Churn Prediction 19th May"  |{                            |{                                       |
|                             |  "classifier_type": "xgb",  |  "classification_report": {            |
|                             |  "stage": "testing"         |    "accuracy": 0.8427358961557664,     |
|                             |}                            |    "exit": {                           |
|                             |                             |      "f1-score": 0.4911147011308562,   |
|                             |                             |      "precision": 0.5914396887159533,  |
|                             |                             |      "recal

<b>Overview:</b>
- use case name : done
- model name : done
- model purpose : done
- model type : done using tags, if existing not satisfactory we can add new
- algorithm used : done using tags
- Flavour used : done
- created by :
- hyperparameters : TB checked, but doable

<b>Datasets used :</b>
- datasets used : done

<b>Model Development metrics:</b>
- Build time metrics:
    - accuracy : done
    - precision : done
    - recall : done
    - f1 : done
    - roc auc : doable
    - log_loss :done
- Class Representation : Done (suppor value in metrics)
- Confusion matrix : Done
- Quality by class : Done

        

### Load model from snowflake repo

In [62]:
# First list all models. This function call returns a snowpark dataframe
model_list = registry.list_models()

In [63]:
model_list.to_pandas()

,CREATION_CONTEXT,CREATION_ENVIRONMENT_SPEC,CREATION_ROLE,CREATION_TIME,ID,INPUT_SPEC,OUTPUT_SPEC,RUNTIME_ENVIRONMENT_SPEC,TYPE,URI,VERSION,DESCRIPTION,METRICS,NAME,TAGS,REGISTRATION_TIMESTAMP
0,None,"{\n ""python"": ""3.8.13""\n}","""ACCOUNTADMIN""",2023-05-05 03:59:15.005000-07:00,df39f65aeb3311edb54d8a8aaf1a29c2,None,None,None,Pipeline,sfc:MODEL_REGISTRY.PUBLIC.SNOWML_MODEL_DF39F65...,None,None,None,"""Diabeties detector""","{\n ""classifier_type"": ""xgb"",\n ""stage"": ""te...",2023-05-05 03:59:18.195000-07:00
1,None,"{\n ""python"": ""3.8.13""\n}","""ACCOUNTADMIN""",2023-05-14 21:05:06.402000-07:00,ac7b0372f2d511ed9bd81a58bc4a8d4c,None,None,None,Pipeline,sfc:MODEL_REGISTRY.PUBLIC.SNOWML_MODEL_AC7B037...,None,None,"{\n ""classification_report"": {\n ""0"": {\n ...","""Diabeties detector""","{\n ""classifier_type"": ""xgb"",\n ""stage"": ""te...",2023-05-14 21:05:09.854000-07:00
2,None,"{\n ""python"": ""3.8.13""\n}","""ACCOUNTADMIN""",2023-05-15 03:14:06.021000-07:00,38dda12af30911ed82411a58bc4a8d4c,None,None,None,Pipeline,sfc:MODEL_REGISTRY.PUBLIC.SNOWML_MODEL_38DDA12...,None,None,"{\n ""classification_report"": {\n ""accuracy...","""Diabeties detector""","{\n ""classifier_type"": ""xgb"",\n ""stage"": ""te...",2023-05-15 03:14:07.916000-07:00
3,None,"{\n ""python"": ""3.8.13""\n}","""ACCOUNTADMIN""",2023-05-19 02:06:34.748000-07:00,73ca7a60f62411ed95000a3279fc73b5,None,None,None,Pipeline,sfc:MODEL_REGISTRY.PUBLIC.SNOWML_MODEL_73CA7A6...,None,None,None,"""Diabeties detector 19th May""","{\n ""classifier_type"": ""xgb"",\n ""stage"": ""te...",2023-05-19 02:06:37.488000-07:00
4,None,"{\n ""python"": ""3.8.13""\n}","""ACCOUNTADMIN""",2023-05-19 02:11:25.920000-07:00,20fb2b08f62511ed95000a3279fc73b5,None,None,None,Pipeline,sfc:MODEL_REGISTRY.PUBLIC.SNOWML_MODEL_20FB2B0...,None,None,"{\n ""classification_report"": {\n ""accuracy...","""Diabeties detector 19th May""","{\n ""classifier_type"": ""xgb"",\n ""stage"": ""te...",2023-05-19 02:11:27.365000-07:00
5,None,"{\n ""python"": ""3.8.13""\n}","""ACCOUNTADMIN""",2023-05-15 03:36:44.798000-07:00,62b26578f30c11ed877cba735c640ef5,None,None,None,Pipeline,sfc:MODEL_REGISTRY.PUBLIC.SNOWML_MODEL_62B2657...,None,None,"{\n ""classification_report"": {\n ""accuracy...","""Diabeties detector""","{\n ""classifier_type"": ""xgb"",\n ""stage"": ""te...",2023-05-15 03:36:46.579000-07:00


In [82]:
model_id = "62b26578f30c11ed877cba735c640ef5"

In [65]:
# need model id for this
ppln_loaded = model_registry.ModelReference(registry=registry, id=model_id)

In [66]:
ppln_loaded

In [67]:
pipeline_restored = registry.load_model(id=model_id)

In [68]:
pipeline_restored

In [69]:
result_new = pipeline_restored.predict(test_df)

SnowparkSessionException: (1404): Cannot perform this operation because the session has been closed.

In [70]:
result_new.show()

NameError: name 'result_new' is not defined

#### Model History 

In [71]:
history = registry.get_model_history(id=model_id).select("*")

In [72]:
history.columns

['EVENT_TIMESTAMP',
 'EVENT_ID',
 'MODEL_ID',
 'ROLE',
 'OPERATION',
 'ATTRIBUTE_NAME',
 '"VALUE[ATTRIBUTE_NAME]"']

In [73]:
history.select("EVENT_TIMESTAMP","ROLE","OPERATION","ATTRIBUTE_NAME").show()

--------------------------------------------------------------------------------------
|"EVENT_TIMESTAMP"                 |"ROLE"          |"OPERATION"  |"ATTRIBUTE_NAME"  |
--------------------------------------------------------------------------------------
|2023-05-15 03:36:46.579000-07:00  |"ACCOUNTADMIN"  |SET          |REGISTRATION      |
|2023-05-15 03:36:47.523000-07:00  |"ACCOUNTADMIN"  |SET          |NAME              |
|2023-05-15 03:36:48.667000-07:00  |"ACCOUNTADMIN"  |SET          |TAGS              |
|2023-05-15 03:44:22.449000-07:00  |"ACCOUNTADMIN"  |SET          |METRICS           |
|2023-05-15 03:44:55.191000-07:00  |"ACCOUNTADMIN"  |SET          |METRICS           |
|2023-05-15 03:45:06.084000-07:00  |"ACCOUNTADMIN"  |SET          |METRICS           |
|2023-05-15 03:45:19.826000-07:00  |"ACCOUNTADMIN"  |SET          |METRICS           |
--------------------------------------------------------------------------------------



In [74]:
from snowflake.ml.model.deployer import LocalDeploymentManager,TargetPlatform, Deployment,Deployer

In [75]:
manager = LocalDeploymentManager()

In [76]:
deployer = Deployer(session,manager)

In [108]:
deployment = deployer.create_deployment("Churn Prediction 19", "/data/models/",TargetPlatform.SNOWPARK,{'relax_version':False})

In [109]:
deployment

Deployment(name='Churn Prediction 19', model_dir_path='/data/models/', platform=<TargetPlatform.SNOWPARK: 'snowpark'>, options={'relax_version': False})

In [110]:
deployer.list_deployments()

[Deployment(name='Churn Prediction', model_dir_path='/data/models/', platform=<TargetPlatform.SNOWPARK: 'snowpark'>, options={'relax_version': False}),
 Deployment(name='Churn Prediction 19', model_dir_path='/data/models/', platform=<TargetPlatform.SNOWPARK: 'snowpark'>, options={'relax_version': False})]

In [111]:
output = deployer.predict('Churn Prediction 19',test_df)

InterfaceError: 252006: Cursor is closed in execute.

In [135]:
session=get_session()

In [142]:
pipeline.get_sklearn_object

'hello'